<a href="https://colab.research.google.com/github/redher97/DheerajReddy_INFO5731_-Fall2021/blob/main/Sentimental_Analysis_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
import io
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import re
# !pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html
# !pip install transformers

import tensorflow as tf

'''# Get the GPU device name.
device_name = tf.test.gpu_device_name()
print(device_name)
# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')
'''
import torch

# If there's a GPU available...
if torch.cuda.is_available():
    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")
CUDA_LAUNCH_BLOCKING=1


# Downloading the csv file from your GitHub account

#data_url = 'https://raw.githubusercontent.com/HuyenNguyenHelen/Drugwar/main/Top5%25.csv'
#def loadFile (url):
#  download = requests.get(url).content
#  df = pd.read_csv(io.StringIO(download.decode('utf-8')))
#  print(df.shape)
#  return df [['created_at', 'author_id','public_metricsretweet_count' , 'public_metricsreply_count', 'public_metricslike_count', 'public_metricsquote_count', 'CleanedTweet', 'hashtags extracted']]
#df = loadFile (data_url)

with open (r'C:\Users\reddy\OneDrive\Desktop\final.csv','r', encoding = 'utf-8') as file:
    df = pd.read_csv(file)
df = df.dropna(subset=['CleanedTweet'])
print(df.head())


# Loading the model
tokenizer = AutoTokenizer.from_pretrained('siebert/sentiment-roberta-large-english')         #'cardiffnlp/twitter-roberta-base-sentiment')           #'nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained ('siebert/sentiment-roberta-large-english')   #('cardiffnlp/twitter-roberta-base-sentiment')               #('nlptown/bert-base-multilingual-uncased-sentiment')
# Defining a sentiment detector
def sentiment_score(tweet):
    tokens = tokenizer.encode(tweet,
                              max_length = 128,
                              truncation=True,
                              pad_to_max_length = True,
                              return_tensors='pt')
    # tokens.to(device)
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

i=0
while i<(len(df)/10):
    m=10*i
    n=m+10
    df_sub = df[m:n]
    print(df_sub['CleanedTweet'])
    df_sub['sentiment'] = df_sub['CleanedTweet'].apply(lambda x: sentiment_score(x))
    df_sub.to_csv(r'C:\Users\reddy\OneDrive\Desktop\whole_sentiment_analysis_roberta.csv', mode='a', header=False)
    print('{}-- done: {}-{}.....................'.format(i, m, n))
    i+=1

In [ ]:

from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
df2 = pd.read_csv(r'C:\Users\reddy\OneDrive\Desktop\whole_sentiment_analysis_roberta.csv')
mask = np.array(Image.open('twitter.png'))
df = df2[df2['sentiment score']==1]

comment_words = ''
stopwords = set(STOPWORDS)


for val in df.CleanedTweet:
	val = str(val)
	tokens = val.split()
	for i in range(len(tokens)):
		tokens[i] = tokens[i].lower()
	comment_words += " ".join(tokens)+" "



wordcloud = WordCloud(stopwords=STOPWORDS, 
               mask=mask, background_color="white",
               max_words=2000, max_font_size=256,
               random_state=42, width=mask.shape[1],
               height=mask.shape[0]).generate(comment_words)
plt.figure(figsize = (20, 20), facecolor = None)
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.show()

In [ ]:

df = df2[df2['sentiment score']==2]

comment_words = ''
stopwords = set(STOPWORDS)


for val in df.CleanedTweet:
	val = str(val)
	tokens = val.split()
	for i in range(len(tokens)):
		tokens[i] = tokens[i].lower()
	
	comment_words += " ".join(tokens)+" "

wordcloud = WordCloud(stopwords=STOPWORDS, 
               mask=mask, background_color="white",
               max_words=2000, max_font_size=256,
               random_state=42, width=mask.shape[1],
               height=mask.shape[0]).generate(comment_words)

# plot the WordCloud image					
plt.figure(figsize = (20,20), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)

plt.show()

In [ ]:
import os,re
import zipfile
import os
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import string
import numpy as np
import matplotlib.pyplot as plt
punctuations= string.punctuation
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
stopword_list = stopwords.words("english")
from nltk.stem.wordnet import WordNetLemmatizer
lem = WordNetLemmatizer()
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk import FreqDist
def clean_text(text,stop_words=True):
    cleaned_text = text.lower() #preprocessing 
    cleaned_text = "".join(c for c in cleaned_text if c not in punctuations) #preprocessing 
    words = cleaned_text.split() 
    if stop_words==True:
        words = [w for w in words if w not in stopword_list if w != '...'] #preprocessing 
        words = [w for w in words if w.isdigit() != True and w !='...']
        words = [lem.lemmatize(word, "v") for word in words]
        words = [lem.lemmatize(word, "n") for word in words] #preprocessing 
        cleaned_text = " ".join(words)
    return cleaned_text
path = r"C:/Users/reddy/OneDrive/Desktop/chiproject/2011-2021/"
directory = os.listdir(path)
searchstring = 'Drugwar'
fg=[]
for fname in directory:
    if os.path.isfile(path + os.sep + fname):
        f = open(path + os.sep + fname, 'r')
        if re.findall(searchstring,f.read()):
            print(path+fname)
            js=pd.read_json(path+fname)
            df=pd.DataFrame([js['tweets'][i]['data'][j] for i in range(len(js['tweets'])) for j in range(len(js['tweets'][i]['data']))])
            df = df.dropna(subset=['text'])
            fg.append(df)
        f.close()
g=[]
df3=fg[0]
for i in range(len(fg)-1):
    df3=fg[i+1][df3.columns]
    g.append(df3)
dt=pd.concat(fg,axis=0)
dt['CleanedTweet']=dt['text'].apply(clean_text)
dt['year']=pd.DatetimeIndex(dt['created_at']).year
df=dt
df.head()

In [ ]:
import seaborn as sns
sns.catplot(x='year',kind='count',data=df)

In [ ]:
df2011=df[df['year']==2011]
df2012=df[df['year']==2012]
df2013=df[df['year']==2013]
df2014=df[df['year']==2014]
df2015=df[df['year']==2015]
df2016=df[df['year']==2016]
df2017=df[df['year']==2017]
df2018=df[df['year']==2018]
df2019=df[df['year']==2019]
df2020=df[df['year']==2020]
df2021=df[df['year']==2021]

In [ ]:
import requests
import pandas as pd
import io
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import re
# !pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html
# !pip install transformers

import tensorflow as tf

'''# Get the GPU device name.
device_name = tf.test.gpu_device_name()
print(device_name)
# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')
'''
import torch

# If there's a GPU available...
if torch.cuda.is_available():
    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")
CUDA_LAUNCH_BLOCKING=1


# Downloading the csv file from your GitHub account

#data_url = 'https://raw.githubusercontent.com/HuyenNguyenHelen/Drugwar/main/Top5%25.csv'
#def loadFile (url):
#  download = requests.get(url).content
#  df = pd.read_csv(io.StringIO(download.decode('utf-8')))
#  print(df.shape)
#  return df [['created_at', 'author_id','public_metricsretweet_count' , 'public_metricsreply_count', 'public_metricslike_count', 'public_metricsquote_count', 'CleanedTweet', 'hashtags extracted']]
#df = loadFile (data_url)


df2011 = df.dropna(subset=['CleanedTweet'])
print(df.head())


# Loading the model
tokenizer = AutoTokenizer.from_pretrained('siebert/sentiment-roberta-large-english')         #'cardiffnlp/twitter-roberta-base-sentiment')           #'nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained ('siebert/sentiment-roberta-large-english')   #('cardiffnlp/twitter-roberta-base-sentiment')               #('nlptown/bert-base-multilingual-uncased-sentiment')
# Defining a sentiment detector
def sentiment_score(tweet):
    tokens = tokenizer.encode(tweet,
                              max_length = 128,
                              truncation=True,
                              pad_to_max_length = True,
                              return_tensors='pt')
    # tokens.to(device)
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

i=0
while i<(len(df)/10):
    m=10*i
    n=m+10
    df_sub = df[m:n]
    print(df_sub['CleanedTweet'])
    df_sub['sentiment'] = df_sub['CleanedTweet'].apply(lambda x: sentiment_score(x))
    df_sub.to_csv(r'C:\Users\reddy\OneDrive\Desktop\2011.csv', mode='a', header=False)
    print('{}-- done: {}-{}.....................'.format(i, m, n))
    i+=1

In [ ]:
import requests
import pandas as pd
import io
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import re
# !pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html
# !pip install transformers

import tensorflow as tf

'''# Get the GPU device name.
device_name = tf.test.gpu_device_name()
print(device_name)
# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')
'''
import torch

# If there's a GPU available...
if torch.cuda.is_available():
    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")
CUDA_LAUNCH_BLOCKING=1


# Downloading the csv file from your GitHub account

#data_url = 'https://raw.githubusercontent.com/HuyenNguyenHelen/Drugwar/main/Top5%25.csv'
#def loadFile (url):
#  download = requests.get(url).content
#  df = pd.read_csv(io.StringIO(download.decode('utf-8')))
#  print(df.shape)
#  return df [['created_at', 'author_id','public_metricsretweet_count' , 'public_metricsreply_count', 'public_metricslike_count', 'public_metricsquote_count', 'CleanedTweet', 'hashtags extracted']]
#df = loadFile (data_url)


df2012 = df.dropna(subset=['CleanedTweet'])
print(df.head())


# Loading the model
tokenizer = AutoTokenizer.from_pretrained('siebert/sentiment-roberta-large-english')         #'cardiffnlp/twitter-roberta-base-sentiment')           #'nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained ('siebert/sentiment-roberta-large-english')   #('cardiffnlp/twitter-roberta-base-sentiment')               #('nlptown/bert-base-multilingual-uncased-sentiment')
# Defining a sentiment detector
def sentiment_score(tweet):
    tokens = tokenizer.encode(tweet,
                              max_length = 128,
                              truncation=True,
                              pad_to_max_length = True,
                              return_tensors='pt')
    # tokens.to(device)
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

i=0
while i<(len(df)/10):
    m=10*i
    n=m+10
    df_sub = df[m:n]
    print(df_sub['CleanedTweet'])
    df_sub['sentiment'] = df_sub['CleanedTweet'].apply(lambda x: sentiment_score(x))
    df_sub.to_csv(r'C:\Users\reddy\OneDrive\Desktop\2012.csv', mode='a', header=False)
    print('{}-- done: {}-{}.....................'.format(i, m, n))
    i+=1

In [ ]:
import requests
import pandas as pd
import io
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import re
# !pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html
# !pip install transformers

import tensorflow as tf

'''# Get the GPU device name.
device_name = tf.test.gpu_device_name()
print(device_name)
# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')
'''
import torch

# If there's a GPU available...
if torch.cuda.is_available():
    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")
CUDA_LAUNCH_BLOCKING=1


# Downloading the csv file from your GitHub account

#data_url = 'https://raw.githubusercontent.com/HuyenNguyenHelen/Drugwar/main/Top5%25.csv'
#def loadFile (url):
#  download = requests.get(url).content
#  df = pd.read_csv(io.StringIO(download.decode('utf-8')))
#  print(df.shape)
#  return df [['created_at', 'author_id','public_metricsretweet_count' , 'public_metricsreply_count', 'public_metricslike_count', 'public_metricsquote_count', 'CleanedTweet', 'hashtags extracted']]
#df = loadFile (data_url)


df2013 = df.dropna(subset=['CleanedTweet'])
print(df.head())


# Loading the model
tokenizer = AutoTokenizer.from_pretrained('siebert/sentiment-roberta-large-english')         #'cardiffnlp/twitter-roberta-base-sentiment')           #'nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained ('siebert/sentiment-roberta-large-english')   #('cardiffnlp/twitter-roberta-base-sentiment')               #('nlptown/bert-base-multilingual-uncased-sentiment')
# Defining a sentiment detector
def sentiment_score(tweet):
    tokens = tokenizer.encode(tweet,
                              max_length = 128,
                              truncation=True,
                              pad_to_max_length = True,
                              return_tensors='pt')
    # tokens.to(device)
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

i=0
while i<(len(df)/10):
    m=10*i
    n=m+10
    df_sub = df[m:n]
    print(df_sub['CleanedTweet'])
    df_sub['sentiment'] = df_sub['CleanedTweet'].apply(lambda x: sentiment_score(x))
    df_sub.to_csv(r'C:\Users\reddy\OneDrive\Desktop\2013.csv', mode='a', header=False)
    print('{}-- done: {}-{}.....................'.format(i, m, n))
    i+=1

In [ ]:
import requests
import pandas as pd
import io
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import re
# !pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html
# !pip install transformers

import tensorflow as tf

'''# Get the GPU device name.
device_name = tf.test.gpu_device_name()
print(device_name)
# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')
'''
import torch

# If there's a GPU available...
if torch.cuda.is_available():
    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")
CUDA_LAUNCH_BLOCKING=1


# Downloading the csv file from your GitHub account

#data_url = 'https://raw.githubusercontent.com/HuyenNguyenHelen/Drugwar/main/Top5%25.csv'
#def loadFile (url):
#  download = requests.get(url).content
#  df = pd.read_csv(io.StringIO(download.decode('utf-8')))
#  print(df.shape)
#  return df [['created_at', 'author_id','public_metricsretweet_count' , 'public_metricsreply_count', 'public_metricslike_count', 'public_metricsquote_count', 'CleanedTweet', 'hashtags extracted']]
#df = loadFile (data_url)


df2014 = df.dropna(subset=['CleanedTweet'])
print(df.head())


# Loading the model
tokenizer = AutoTokenizer.from_pretrained('siebert/sentiment-roberta-large-english')         #'cardiffnlp/twitter-roberta-base-sentiment')           #'nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained ('siebert/sentiment-roberta-large-english')   #('cardiffnlp/twitter-roberta-base-sentiment')               #('nlptown/bert-base-multilingual-uncased-sentiment')
# Defining a sentiment detector
def sentiment_score(tweet):
    tokens = tokenizer.encode(tweet,
                              max_length = 128,
                              truncation=True,
                              pad_to_max_length = True,
                              return_tensors='pt')
    # tokens.to(device)
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

i=0
while i<(len(df)/10):
    m=10*i
    n=m+10
    df_sub = df[m:n]
    print(df_sub['CleanedTweet'])
    df_sub['sentiment'] = df_sub['CleanedTweet'].apply(lambda x: sentiment_score(x))
    df_sub.to_csv(r'C:\Users\reddy\OneDrive\Desktop\2014.csv', mode='a', header=False)
    print('{}-- done: {}-{}.....................'.format(i, m, n))
    i+=1

In [ ]:
import requests
import pandas as pd
import io
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import re
# !pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html
# !pip install transformers

import tensorflow as tf

'''# Get the GPU device name.
device_name = tf.test.gpu_device_name()
print(device_name)
# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')
'''
import torch

# If there's a GPU available...
if torch.cuda.is_available():
    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")
CUDA_LAUNCH_BLOCKING=1


# Downloading the csv file from your GitHub account

#data_url = 'https://raw.githubusercontent.com/HuyenNguyenHelen/Drugwar/main/Top5%25.csv'
#def loadFile (url):
#  download = requests.get(url).content
#  df = pd.read_csv(io.StringIO(download.decode('utf-8')))
#  print(df.shape)
#  return df [['created_at', 'author_id','public_metricsretweet_count' , 'public_metricsreply_count', 'public_metricslike_count', 'public_metricsquote_count', 'CleanedTweet', 'hashtags extracted']]
#df = loadFile (data_url)


df2015 = df.dropna(subset=['CleanedTweet'])
print(df.head())


# Loading the model
tokenizer = AutoTokenizer.from_pretrained('siebert/sentiment-roberta-large-english')         #'cardiffnlp/twitter-roberta-base-sentiment')           #'nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained ('siebert/sentiment-roberta-large-english')   #('cardiffnlp/twitter-roberta-base-sentiment')               #('nlptown/bert-base-multilingual-uncased-sentiment')
# Defining a sentiment detector
def sentiment_score(tweet):
    tokens = tokenizer.encode(tweet,
                              max_length = 128,
                              truncation=True,
                              pad_to_max_length = True,
                              return_tensors='pt')
    # tokens.to(device)
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

i=0
while i<(len(df)/10):
    m=10*i
    n=m+10
    df_sub = df[m:n]
    print(df_sub['CleanedTweet'])
    df_sub['sentiment'] = df_sub['CleanedTweet'].apply(lambda x: sentiment_score(x))
    df_sub.to_csv(r'C:\Users\reddy\OneDrive\Desktop\2015.csv', mode='a', header=False)
    print('{}-- done: {}-{}.....................'.format(i, m, n))
    i+=1

In [ ]:
import requests
import pandas as pd
import io
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import re
# !pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html
# !pip install transformers

import tensorflow as tf

'''# Get the GPU device name.
device_name = tf.test.gpu_device_name()
print(device_name)
# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')
'''
import torch

# If there's a GPU available...
if torch.cuda.is_available():
    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")
CUDA_LAUNCH_BLOCKING=1


# Downloading the csv file from your GitHub account

#data_url = 'https://raw.githubusercontent.com/HuyenNguyenHelen/Drugwar/main/Top5%25.csv'
#def loadFile (url):
#  download = requests.get(url).content
#  df = pd.read_csv(io.StringIO(download.decode('utf-8')))
#  print(df.shape)
#  return df [['created_at', 'author_id','public_metricsretweet_count' , 'public_metricsreply_count', 'public_metricslike_count', 'public_metricsquote_count', 'CleanedTweet', 'hashtags extracted']]
#df = loadFile (data_url)


df2016 = df.dropna(subset=['CleanedTweet'])
print(df.head())


# Loading the model
tokenizer = AutoTokenizer.from_pretrained('siebert/sentiment-roberta-large-english')         #'cardiffnlp/twitter-roberta-base-sentiment')           #'nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained ('siebert/sentiment-roberta-large-english')   #('cardiffnlp/twitter-roberta-base-sentiment')               #('nlptown/bert-base-multilingual-uncased-sentiment')
# Defining a sentiment detector
def sentiment_score(tweet):
    tokens = tokenizer.encode(tweet,
                              max_length = 128,
                              truncation=True,
                              pad_to_max_length = True,
                              return_tensors='pt')
    # tokens.to(device)
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

i=0
while i<(len(df)/10):
    m=10*i
    n=m+10
    df_sub = df[m:n]
    print(df_sub['CleanedTweet'])
    df_sub['sentiment'] = df_sub['CleanedTweet'].apply(lambda x: sentiment_score(x))
    df_sub.to_csv(r'C:\Users\reddy\OneDrive\Desktop\2016.csv', mode='a', header=False)
    print('{}-- done: {}-{}.....................'.format(i, m, n))
    i+=1

In [ ]:
import requests
import pandas as pd
import io
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import re
# !pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html
# !pip install transformers

import tensorflow as tf

'''# Get the GPU device name.
device_name = tf.test.gpu_device_name()
print(device_name)
# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')
'''
import torch

# If there's a GPU available...
if torch.cuda.is_available():
    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")
CUDA_LAUNCH_BLOCKING=1


# Downloading the csv file from your GitHub account

#data_url = 'https://raw.githubusercontent.com/HuyenNguyenHelen/Drugwar/main/Top5%25.csv'
#def loadFile (url):
#  download = requests.get(url).content
#  df = pd.read_csv(io.StringIO(download.decode('utf-8')))
#  print(df.shape)
#  return df [['created_at', 'author_id','public_metricsretweet_count' , 'public_metricsreply_count', 'public_metricslike_count', 'public_metricsquote_count', 'CleanedTweet', 'hashtags extracted']]
#df = loadFile (data_url)


df2017 = df.dropna(subset=['CleanedTweet'])
print(df.head())


# Loading the model
tokenizer = AutoTokenizer.from_pretrained('siebert/sentiment-roberta-large-english')         #'cardiffnlp/twitter-roberta-base-sentiment')           #'nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained ('siebert/sentiment-roberta-large-english')   #('cardiffnlp/twitter-roberta-base-sentiment')               #('nlptown/bert-base-multilingual-uncased-sentiment')
# Defining a sentiment detector
def sentiment_score(tweet):
    tokens = tokenizer.encode(tweet,
                              max_length = 128,
                              truncation=True,
                              pad_to_max_length = True,
                              return_tensors='pt')
    # tokens.to(device)
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

i=0
while i<(len(df)/10):
    m=10*i
    n=m+10
    df_sub = df[m:n]
    print(df_sub['CleanedTweet'])
    df_sub['sentiment'] = df_sub['CleanedTweet'].apply(lambda x: sentiment_score(x))
    df_sub.to_csv(r'C:\Users\reddy\OneDrive\Desktop\2017.csv', mode='a', header=False)
    print('{}-- done: {}-{}.....................'.format(i, m, n))
    i+=1

In [ ]:
import requests
import pandas as pd
import io
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import re
# !pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html
# !pip install transformers

import tensorflow as tf

'''# Get the GPU device name.
device_name = tf.test.gpu_device_name()
print(device_name)
# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')
'''
import torch

# If there's a GPU available...
if torch.cuda.is_available():
    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")
CUDA_LAUNCH_BLOCKING=1


# Downloading the csv file from your GitHub account

#data_url = 'https://raw.githubusercontent.com/HuyenNguyenHelen/Drugwar/main/Top5%25.csv'
#def loadFile (url):
#  download = requests.get(url).content
#  df = pd.read_csv(io.StringIO(download.decode('utf-8')))
#  print(df.shape)
#  return df [['created_at', 'author_id','public_metricsretweet_count' , 'public_metricsreply_count', 'public_metricslike_count', 'public_metricsquote_count', 'CleanedTweet', 'hashtags extracted']]
#df = loadFile (data_url)


df2018 = df.dropna(subset=['CleanedTweet'])
print(df.head())


# Loading the model
tokenizer = AutoTokenizer.from_pretrained('siebert/sentiment-roberta-large-english')         #'cardiffnlp/twitter-roberta-base-sentiment')           #'nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained ('siebert/sentiment-roberta-large-english')   #('cardiffnlp/twitter-roberta-base-sentiment')               #('nlptown/bert-base-multilingual-uncased-sentiment')
# Defining a sentiment detector
def sentiment_score(tweet):
    tokens = tokenizer.encode(tweet,
                              max_length = 128,
                              truncation=True,
                              pad_to_max_length = True,
                              return_tensors='pt')
    # tokens.to(device)
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

i=0
while i<(len(df)/10):
    m=10*i
    n=m+10
    df_sub = df[m:n]
    print(df_sub['CleanedTweet'])
    df_sub['sentiment'] = df_sub['CleanedTweet'].apply(lambda x: sentiment_score(x))
    df_sub.to_csv(r'C:\Users\reddy\OneDrive\Desktop\2018.csv', mode='a', header=False)
    print('{}-- done: {}-{}.....................'.format(i, m, n))
    i+=1

In [ ]:
import requests
import pandas as pd
import io
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import re
# !pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html
# !pip install transformers

import tensorflow as tf

'''# Get the GPU device name.
device_name = tf.test.gpu_device_name()
print(device_name)
# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')
'''
import torch

# If there's a GPU available...
if torch.cuda.is_available():
    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")
CUDA_LAUNCH_BLOCKING=1


# Downloading the csv file from your GitHub account

#data_url = 'https://raw.githubusercontent.com/HuyenNguyenHelen/Drugwar/main/Top5%25.csv'
#def loadFile (url):
#  download = requests.get(url).content
#  df = pd.read_csv(io.StringIO(download.decode('utf-8')))
#  print(df.shape)
#  return df [['created_at', 'author_id','public_metricsretweet_count' , 'public_metricsreply_count', 'public_metricslike_count', 'public_metricsquote_count', 'CleanedTweet', 'hashtags extracted']]
#df = loadFile (data_url)


df2019 = df.dropna(subset=['CleanedTweet'])
print(df.head())


# Loading the model
tokenizer = AutoTokenizer.from_pretrained('siebert/sentiment-roberta-large-english')         #'cardiffnlp/twitter-roberta-base-sentiment')           #'nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained ('siebert/sentiment-roberta-large-english')   #('cardiffnlp/twitter-roberta-base-sentiment')               #('nlptown/bert-base-multilingual-uncased-sentiment')
# Defining a sentiment detector
def sentiment_score(tweet):
    tokens = tokenizer.encode(tweet,
                              max_length = 128,
                              truncation=True,
                              pad_to_max_length = True,
                              return_tensors='pt')
    # tokens.to(device)
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

i=0
while i<(len(df)/10):
    m=10*i
    n=m+10
    df_sub = df[m:n]
    print(df_sub['CleanedTweet'])
    df_sub['sentiment'] = df_sub['CleanedTweet'].apply(lambda x: sentiment_score(x))
    df_sub.to_csv(r'C:\Users\reddy\OneDrive\Desktop\2019.csv', mode='a', header=False)
    print('{}-- done: {}-{}.....................'.format(i, m, n))
    i+=1

In [ ]:
import requests
import pandas as pd
import io
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import re
# !pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html
# !pip install transformers

import tensorflow as tf

'''# Get the GPU device name.
device_name = tf.test.gpu_device_name()
print(device_name)
# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')
'''
import torch

# If there's a GPU available...
if torch.cuda.is_available():
    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")
CUDA_LAUNCH_BLOCKING=1


# Downloading the csv file from your GitHub account

#data_url = 'https://raw.githubusercontent.com/HuyenNguyenHelen/Drugwar/main/Top5%25.csv'
#def loadFile (url):
#  download = requests.get(url).content
#  df = pd.read_csv(io.StringIO(download.decode('utf-8')))
#  print(df.shape)
#  return df [['created_at', 'author_id','public_metricsretweet_count' , 'public_metricsreply_count', 'public_metricslike_count', 'public_metricsquote_count', 'CleanedTweet', 'hashtags extracted']]
#df = loadFile (data_url)


df2020 = df.dropna(subset=['CleanedTweet'])
print(df.head())


# Loading the model
tokenizer = AutoTokenizer.from_pretrained('siebert/sentiment-roberta-large-english')         #'cardiffnlp/twitter-roberta-base-sentiment')           #'nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained ('siebert/sentiment-roberta-large-english')   #('cardiffnlp/twitter-roberta-base-sentiment')               #('nlptown/bert-base-multilingual-uncased-sentiment')
# Defining a sentiment detector
def sentiment_score(tweet):
    tokens = tokenizer.encode(tweet,
                              max_length = 128,
                              truncation=True,
                              pad_to_max_length = True,
                              return_tensors='pt')
    # tokens.to(device)
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

i=0
while i<(len(df)/10):
    m=10*i
    n=m+10
    df_sub = df[m:n]
    print(df_sub['CleanedTweet'])
    df_sub['sentiment'] = df_sub['CleanedTweet'].apply(lambda x: sentiment_score(x))
    df_sub.to_csv(r'C:\Users\reddy\OneDrive\Desktop\2020.csv', mode='a', header=False)
    print('{}-- done: {}-{}.....................'.format(i, m, n))
    i+=1

In [ ]:
import requests
import pandas as pd
import io
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import re
# !pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html
# !pip install transformers

import tensorflow as tf

'''# Get the GPU device name.
device_name = tf.test.gpu_device_name()
print(device_name)
# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')
'''
import torch

# If there's a GPU available...
if torch.cuda.is_available():
    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")
CUDA_LAUNCH_BLOCKING=1


# Downloading the csv file from your GitHub account

#data_url = 'https://raw.githubusercontent.com/HuyenNguyenHelen/Drugwar/main/Top5%25.csv'
#def loadFile (url):
#  download = requests.get(url).content
#  df = pd.read_csv(io.StringIO(download.decode('utf-8')))
#  print(df.shape)
#  return df [['created_at', 'author_id','public_metricsretweet_count' , 'public_metricsreply_count', 'public_metricslike_count', 'public_metricsquote_count', 'CleanedTweet', 'hashtags extracted']]
#df = loadFile (data_url)


df2021 = df.dropna(subset=['CleanedTweet'])
print(df.head())


# Loading the model
tokenizer = AutoTokenizer.from_pretrained('siebert/sentiment-roberta-large-english')         #'cardiffnlp/twitter-roberta-base-sentiment')           #'nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained ('siebert/sentiment-roberta-large-english')   #('cardiffnlp/twitter-roberta-base-sentiment')               #('nlptown/bert-base-multilingual-uncased-sentiment')
# Defining a sentiment detector
def sentiment_score(tweet):
    tokens = tokenizer.encode(tweet,
                              max_length = 128,
                              truncation=True,
                              pad_to_max_length = True,
                              return_tensors='pt')
    # tokens.to(device)
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

i=0
while i<(len(df)/10):
    m=10*i
    n=m+10
    df_sub = df[m:n]
    print(df_sub['CleanedTweet'])
    df_sub['sentiment'] = df_sub['CleanedTweet'].apply(lambda x: sentiment_score(x))
    df_sub.to_csv(r'C:\Users\reddy\OneDrive\Desktop\2021.csv', mode='a', header=False)
    print('{}-- done: {}-{}.....................'.format(i, m, n))
    i+=1

In [ ]:
import pandas as pd
import numpy as np
df1 = pd.read_csv('/content/20'+'11'+'.csv',header=None)
column=['n','lang', 'public_metrics', 'created_at', 'author_id', 'text', 'id',
       'geo', 'withheld', 'CleanedTweet', 'year','sentiment']
df1.columns=column
fg=[df1]
for i in range(12,22):
  df2 = pd.read_csv('/content/20'+str(i)+'.csv')
  df2.columns=column
  fg.append(df2)
df1=pd.concat(fg,axis=0)
df1.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.catplot(x='year',data=df1,kind='count',hue='sentiment',aspect=2)